In [ ]:
## STANDARD IMPORTS
import os
import time
import h5py
import numpy as np
import numpy.ma as ma
import pandas as pd
import json

# analysis
import scipy.stats
from scipy.optimize import curve_fit

# data handling
!pip3 install ../h5flow
import h5flow
from h5flow.data import dereference

## 3D PLOTTING
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm, colors
import matplotlib.patches as mpatches
from matplotlib.colors import BoundaryNorm

In [ ]:
dirname = 'mc_processed_MiniRun5_1E19_RHC.flow.0000000.FLOW_TrapType_evts_all'
spes_filename = dirname+'/spes_evt.npz'
noise_filename = dirname+'/noise_evt.npz'
config_filename = dirname+'/config.json'
hits_filename = dirname+'/hits_evt.npz'
hits_config_filename = dirname+'/hits_config.json'
true_hits_filename = dirname+'/true_nu_int.csv'

# check if true hits file exists
if os.path.exists(true_hits_filename):
    true_hits = pd.read_csv(true_hits_filename)
    print(true_hits.head())

# print configuration and hit configuration
with open(config_filename) as json_file:
    config = json.load(json_file)
    print(json.dumps(config, indent=4))
with open(hits_config_filename) as json_file:
    hits_config = json.load(json_file)
    print(json.dumps(hits_config, indent=4))

In [3]:
import scipy.stats

def clopper_pearson_interval(k, n, alpha=0.6827):
    alpha = 1 - alpha
    lo = scipy.stats.beta.ppf(alpha / 2, k, n - k + 1) if k > 0 else 0.0
    hi = scipy.stats.beta.ppf(1 - alpha / 2, k + 1, n - k) if k < n else 1.0
    return lo, hi


# gaussian fit func
def gaussian(x, A, mu, sig):
    return A * np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [4]:
# load true hits
true_hits = pd.read_csv(true_hits_filename)
true_hit_idxs = true_hits['start_time_idx'].values

# load ticks histogram of all hits
hits_file = np.load(hits_filename)
hits_arr = hits_file['arr_0']
flat_hits = np.sum(hits_arr, axis=(0,1))

In [ ]:
tolerance = 6

# info by trap type (ArClight)
acl_rec_true_hits = 0
acl_true_hits = 0
acl_rec_hits = 0
acl_delta_t = []

# info by trap type (LCM)
lcm_rec_true_hits = 0
lcm_true_hits = 0
lcm_rec_hits = 0
lcm_delta_t = []

# loop over events
for i_evt_lrs in range(hits_arr.shape[0]):

    # loop over traps
    for i_trap in range(hits_arr.shape[1]):
        is_acl = i_trap % 2 == 0
        i_tpc = i_trap // 2

        hits = np.where(hits_arr[i_evt_lrs, i_trap])[0]
        true_hit_idxs_tpc = true_hits[(true_hits['event_id'] == i_evt_lrs) & (true_hits['tpc_num'] == i_tpc)]['start_time_idx'].values

        # ACLs
        if is_acl:

            # add true hits
            acl_true_hits += len(true_hit_idxs_tpc)

            # add rec hits
            for i_hit in hits:
                acl_rec_hits += 1

                # check if true hit is within tolerance in this specific tpc

                # get true hit indices within tolerance of this hit
                delta_ts = (i_hit - true_hit_idxs_tpc)
                delta_ts = delta_ts[(delta_ts < tolerance) & (delta_ts > 0)]
                if delta_ts.size > 0:
                    delta_ts = min(delta_ts)
                    acl_delta_t.append(delta_ts)

                    # add to true rec hits
                    acl_rec_true_hits += 1

        else:

            # add true hits
            lcm_true_hits += len(true_hit_idxs_tpc)

            # add rec hits
            for i_hit in hits:
                lcm_rec_hits += 1

                # get true hit indices within tolerance of this hit
                delta_ts = (i_hit - true_hit_idxs_tpc)
                delta_ts = delta_ts[(delta_ts < tolerance) & (delta_ts > 0)]
                if delta_ts.size > 0:
                    delta_ts = min(delta_ts)
                    lcm_delta_t.append(delta_ts)

                    # add to true rec hits
                    lcm_rec_true_hits += 1



# caluclate efficiency +/- clopper pearson
print(f'ACL True Hits: {acl_true_hits}')
print(f'ACL Rec Hits: {acl_rec_hits}')
print(f'ACL Rec True Hits: {acl_rec_true_hits}')

print(f'LCM True Hits: {lcm_true_hits}')
print(f'LCM Rec Hits: {lcm_rec_hits}')
print(f'LCM Rec True Hits: {lcm_rec_true_hits}')

# ACL efficiency
acl_eff = acl_rec_true_hits / acl_true_hits
acl_eff_err = clopper_pearson_interval(acl_rec_true_hits, acl_true_hits)
print(f'ACL Efficiency: {acl_eff:.2f} + {acl_eff_err[1] - acl_eff:.2f} - {acl_eff - acl_eff_err[0]:.2f}')
# ACL fake rate
acl_fake_rate = 1 - (acl_rec_true_hits / acl_rec_hits)
acl_fake_rate_err = clopper_pearson_interval(acl_rec_true_hits, acl_rec_hits)
print(f'ACL Fake Rate: {acl_fake_rate:.2f} + {1 - acl_fake_rate_err[0] - acl_fake_rate:.2f} - {1 - acl_fake_rate_err[1] - acl_fake_rate:.2f}')

# LCM efficiency
lcm_eff = lcm_rec_true_hits / lcm_true_hits
lcm_eff_err = clopper_pearson_interval(lcm_rec_true_hits, lcm_true_hits)
print(f'LCM Efficiency: {lcm_eff:.2f} + {lcm_eff_err[1] - lcm_eff:.2f} - {lcm_eff - lcm_eff_err[0]:.2f}')

# LCM fake rate
lcm_fake_rate = 1 - (lcm_rec_true_hits / lcm_rec_hits)
lcm_fake_rate_err = clopper_pearson_interval(lcm_rec_true_hits, lcm_rec_hits)
print(f'LCM Fake Rate: {lcm_fake_rate:.2f} + {1 - lcm_fake_rate_err[0] - lcm_fake_rate:.2f} - {1 - lcm_fake_rate_err[1] - lcm_fake_rate:.2f}')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Define Gaussian function
def gaussian(x, A, mu, sig):
    return A * np.exp(-0.5 * ((x - mu) / sig) ** 2)

# Set up figure with GridSpec
fig, ax = plt.subplots(2, 2, figsize=(8, 5), gridspec_kw={'height_ratios': [3, 1]})
nbins = 10
#tolerance = 10
interp_bin_centres = np.linspace(0, tolerance, 1000)

# Function to fit and plot
def plot_histogram_and_fit(ax_main, ax_resid, delta_t, color, label):
    # Histogram
    ax_main.hist(delta_t, bins=nbins, range=(0, tolerance), histtype='step', color=color, label=label)

    # Bin calculations
    hist, bin_edges = np.histogram(delta_t, bins=nbins, range=(0, tolerance))
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    sqrtN = np.sqrt(hist)

    # Error bars
    ax_main.errorbar(bin_centers, hist, yerr=sqrtN, fmt='none', color=color, alpha=0.5)

    # Fit Gaussian
    param_bounds = ([0, np.min(bin_centers), 0.01], [np.inf, np.max(bin_centers), np.std(delta_t)])
    popt, _ = curve_fit(gaussian, bin_centers, hist, p0=[max(hist), np.mean(delta_t), np.std(delta_t)], bounds=param_bounds, maxfev=10000)

    # Plot fit
    ax_main.plot(interp_bin_centres, gaussian(interp_bin_centres, *popt), color='k', linestyle='--', label=f'{label} fit')

    # Labels
    #ax_main.set_xlabel('Delta T (ticks)')
    ax_main.set_ylabel('Counts')
    ax_main.set_title(f'{label} Delta T')
    ax_main.text(0.5, 0.9, f'Mean (ns): {popt[1]*16:.2f}', transform=ax_main.transAxes)
    ax_main.text(0.5, 0.85, f'Sigma (ns): {popt[2]*16:.2f}', transform=ax_main.transAxes)
    ax_main.legend()

    # Residuals
    resids = (hist - gaussian(bin_centers, *popt))
    ax_resid.step(bin_centers, resids/sqrtN, color=color)
    ax_resid.fill_between(bin_centers, (resids-sqrtN)/sqrtN, (resids+sqrtN)/sqrtN, color=color, alpha=0.5, step='pre')
    ax_resid.axhline(0, color='k', linestyle='--')
    ax_resid.set_xlabel('Delta T (ticks)')
    ax_resid.set_ylabel(r'($\sigma$)')


# Plot ACL
plot_histogram_and_fit(ax[0, 0], ax[1, 0], acl_delta_t, 'b', 'ACL')

# Plot LCM
plot_histogram_and_fit(ax[0, 1], ax[1, 1], lcm_delta_t, 'r', 'LCM')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Define Landau function
def landau_func(x, mpv, eta, A):
  return A * np.exp(-0.5 * ((x - mpv) / eta + np.exp(-(x - mpv) / eta)))

# Set up figure with GridSpec
fig, ax = plt.subplots(2, 2, figsize=(8, 5), gridspec_kw={'height_ratios': [3, 1]})
nbins = 10
#tolerance = 6
interp_bin_centres = np.linspace(0, tolerance, 1000)

# Function to fit and plot
def plot_histogram_and_fit(ax_main, ax_resid, delta_t, color, label):
  # Histogram
  ax_main.hist(delta_t, bins=nbins, range=(0, tolerance), histtype='step', color=color, label=label)

  # Bin calculations
  hist, bin_edges = np.histogram(delta_t, bins=nbins, range=(0, tolerance))
  bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
  sqrtN = np.sqrt(hist)

  # Error bars
  ax_main.errorbar(bin_centers, hist, yerr=sqrtN, fmt='none', color=color, alpha=0.5)

  # Fit Landau
  param_bounds = ([0, 0, 0], [np.inf, np.inf, np.inf])
  popt, _ = curve_fit(landau_func, bin_centers, hist, p0=[np.mean(delta_t), np.std(delta_t), max(hist)], maxfev=10000)

  # Plot fit
  ax_main.plot(interp_bin_centres, landau_func(interp_bin_centres, *popt), color='k', linestyle='--', label=f'{label} fit')

  # Labels
  ax_main.set_ylabel('Counts')
  ax_main.set_title(f'{label} Delta T')
  #ax_main.text(0.5, 0.9, f'MPV (ns): {popt[0]*16:.2f}', transform=ax_main.transAxes)
  #ax_main.text(0.5, 0.85, f'Eta (ns): {popt[1]*16:.2f}', transform=ax_main.transAxes)
  # sigma = eta * 1.71
  ax_main.text(0.5, 0.8, f'Sigma_eta (ns): {popt[1]*16*1.71:.2f}', transform=ax_main.transAxes)
  # fwhm = eta * 4.018
  #ax_main.text(0.5, 0.75, f'FWHM_eta (ns): {popt[1]*16*4.018:.2f}', transform=ax_main.transAxes)
  # fwhm from plot
  max_val = np.max(landau_func(interp_bin_centres, *popt))
  fwhm = interp_bin_centres[np.where(landau_func(interp_bin_centres, *popt) > max_val/2)]
  fwhm = fwhm[-1] - fwhm[0]
  #ax_main.text(0.5, 0.7, f'FWHM_calc (ns): {fwhm*16:.2f}', transform=ax_main.transAxes)
  # sigma = 0.425 * FWHM
  ax_main.text(0.5, 0.65, f'Sigma_fwhm (ns): {0.425*fwhm*16:.2f}', transform=ax_main.transAxes)
  ax_main.legend()

  # Residuals
  resids = (hist - landau_func(bin_centers, *popt))
  ax_resid.step(bin_centers, resids/sqrtN, color=color)
  ax_resid.fill_between(bin_centers, (resids-sqrtN)/sqrtN, (resids+sqrtN)/sqrtN, color=color, alpha=0.5, step='pre')
  ax_resid.axhline(0, color='k', linestyle='--')
  ax_resid.set_xlabel('Delta T (ticks)')
  ax_resid.set_ylabel(r'($\sigma$)')

# Plot ACL
plot_histogram_and_fit(ax[0, 0], ax[1, 0], acl_delta_t, 'b', 'ACL')

# Plot LCM
plot_histogram_and_fit(ax[0, 1], ax[1, 1], lcm_delta_t, 'r', 'LCM')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Define convolution of Gaussian and Landau function
from scipy.special import erf

def gauss_landau_conv(x, mpv, eta, A, sigma):
  return A * np.exp((eta / (2 * sigma**2)) * (2 * mpv + eta)) * np.exp(-x * eta / sigma**2) * \
       (1 + erf((x - mpv - eta) / (np.sqrt(2) * sigma)))

# Function to fit and plot
def plot_histogram_and_fit(ax_main, ax_resid, delta_t, color, label):
  # Histogram
  ax_main.hist(delta_t, bins=nbins, range=(0, tolerance), histtype='step', color=color, label=label)

  # Bin calculations
  hist, bin_edges = np.histogram(delta_t, bins=nbins, range=(0, tolerance))
  bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
  sqrtN = np.sqrt(hist)

  # Error bars
  ax_main.errorbar(bin_centers, hist, yerr=sqrtN, fmt='none', color=color, alpha=0.5)

  # Fit Gaussian-Landau convolution
  param_bounds = ([0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf])
  popt, _ = curve_fit(gauss_landau_conv, bin_centers, hist, p0=[np.mean(delta_t), np.std(delta_t), max(hist), np.std(delta_t)], bounds=param_bounds, maxfev=10000)

  # Plot fit
  ax_main.plot(interp_bin_centres, gauss_landau_conv(interp_bin_centres, *popt), color='k', linestyle='--', label=f'{label} fit')

  # Labels
  ax_main.set_ylabel('Counts')
  ax_main.legend()

  # Residuals
  resids = (hist - gauss_landau_conv(bin_centers, *popt))
  ax_resid.step(bin_centers, resids/sqrtN, color=color)
  ax_resid.fill_between(bin_centers, (resids-sqrtN)/sqrtN, (resids+sqrtN)/sqrtN, color=color, alpha=0.5, step='pre')
  ax_resid.axhline(0, color='k', linestyle='--')
  ax_resid.set_xlabel('Delta T (ticks)')
  ax_resid.set_ylabel(r'($\sigma$)')

  # Print fit parameters on the plot
  ax_main.text(0.5, 0.9, f'MPV (ns): {popt[0]*16:.2f}', transform=ax_main.transAxes)
  ax_main.text(0.5, 0.85, f'Eta (ns): {popt[1]*16:.2f}', transform=ax_main.transAxes)
  # sigma from eta
  ax_main.text(0.5, 0.75, f'Sigma_eta (ns): {popt[1]*16*1.71:.2f}', transform=ax_main.transAxes)
  # sigma from fit
  ax_main.text(0.5, 0.70, f'Sigma (ns): {popt[3]*16:.2f}', transform=ax_main.transAxes)
  # sigma from fwhm
  max_val = np.max(gauss_landau_conv(interp_bin_centres, *popt))
  fwhm = interp_bin_centres[np.where(gauss_landau_conv(interp_bin_centres, *popt) > max_val/2)]
  fwhm = fwhm[-1] - fwhm[0]
  ax_main.text(0.5, 0.65, f'Sigma_fwhm (ns): {0.425*fwhm*16:.2f}', transform=ax_main.transAxes)


# Set up figure with GridSpec
fig, ax = plt.subplots(2, 2, figsize=(8, 5), gridspec_kw={'height_ratios': [3, 1]})
interp_bin_centres = np.linspace(0, tolerance, 1000)

# Plot ACL
plot_histogram_and_fit(ax[0, 0], ax[1, 0], acl_delta_t, 'b', 'ACL')

# Plot LCM
plot_histogram_and_fit(ax[0, 1], ax[1, 1], lcm_delta_t, 'r', 'LCM')

# Adjust layout
plt.tight_layout()
plt.show()

In [9]:
def plot_waveform(i_evt_lrs, time_bins, wvfm, hit_where, i_mask, true_hits, logy=False):

  fig = plt.figure(figsize=(12, 3))
  # set a title
  fig.suptitle(f'Event {i_evt_lrs}, TPC {i_mask[0] // 2}')

  if logy==False:
  # Linear plot
    # acls
    plt.plot(time_bins, wvfm[0], label='ACL', color='red', alpha=0.5)
    plt.plot(hit_where[0] * 16/1000, wvfm[0][hit_where[0]], 'x', color='red')
    # lcms
    plt.plot(time_bins, wvfm[1], label='LCMS', color='blue', alpha=0.5)
    plt.plot(hit_where[1] * 16/1000, wvfm[1][hit_where[1]], 'x', color='blue')
    # baseline
    plt.axhline(0, color='k', linestyle=':', alpha=0.5)
    # true hits
    for hit_idx in true_hits:
      plt.axvline(hit_idx * 16/1000, color='purple', linestyle=':', alpha=0.5, label='True hits')
    plt.xlabel('Time (us)')
    plt.ylabel('SPEs')
    plt.legend()

  else:
  # Logarithmic plot
    # acls
    plt.plot(time_bins, wvfm[0] + 1, label='ACL', color='red', alpha=0.5)
    plt.plot(hit_where[0] * 16/1000, wvfm[0][hit_where[0]] + 1, 'x', color='red')
    # lcms
    plt.plot(time_bins, wvfm[1] + 1, label='LCMS', color='blue', alpha=0.5)
    plt.plot(hit_where[1] * 16/1000, wvfm[1][hit_where[1]] + 1, 'x', color='blue')
    # baseline
    plt.axhline(1, color='k', linestyle=':', alpha=0.5)
    # true hits
    for hit_idx in true_hits:
      plt.axvline(hit_idx * 16/1000, color='purple', linestyle=':', alpha=0.5)
    plt.yscale('log')
    plt.xlabel('Time (us)')

  plt.tight_layout()
  plt.show()

In [10]:
from scipy.ndimage import uniform_filter1d

# load hit finder config
with open(hits_config_filename) as json_file:
    hits_config = json.load(json_file)
n_noise_factor = hits_config['n_noise_factor']
n_bins_rolled = hits_config['n_bins_rolled']
n_sqrt_rt_factor = hits_config['n_sqrt_rt_factor']
pe_weight = hits_config['pe_weight']

# loading events
time_bins = np.linspace(0, 16, 1000)

# wvfms
summed_wvfms_file = np.load(spes_filename)
summed_wvfms = np.array(summed_wvfms_file['arr_0'])

# noise thresholds
heights_file = np.load(noise_filename)
heights = np.array(heights_file['arr_0'])

# loading hits
hits_file = np.load(hits_filename)
hits = np.array(hits_file['arr_0'])

# loading true hits
true_hits = pd.read_csv(true_hits_filename)

In [ ]:
# plotting script to plot the summed traptype waveforms per tpc with interactions
def plot_summed_trap_waveform_per_tpc_with_interactions(i_evt, time_bins, summed_wvfms, hits, true_hits):

  # 2x2 TPC subplot grid, each with traptype sum waveforms
  fig, axs = plt.subplots(8, 1, figsize=(15, 15))

  for tpc in range(8):
    acl_mask = 2*tpc
    lcm_mask = 2*tpc + 1

    axs[tpc].axhline(0, color='k', linestyle=':')

    # get wvfm, threshold, and hits for the event
    acl_wvfm = summed_wvfms[i_evt, acl_mask]
    acl_hit = hits[i_evt, acl_mask]
    acl_hit_where = np.where(acl_hit)[0]

    lcm_wvfm = summed_wvfms[i_evt, lcm_mask]
    lcm_hit = hits[i_evt, lcm_mask]
    lcm_hit_where = np.where(lcm_hit)[0]

    axs[tpc].plot(time_bins, acl_wvfm, label='ACL', color='red', alpha=0.5)
    axs[tpc].plot(acl_hit_where * 16/1000, acl_wvfm[acl_hit_where], 'x', color='red', label='ACL hits')

    axs[tpc].plot(time_bins, lcm_wvfm, label='LCM', color='blue', alpha=0.5)
    axs[tpc].plot(lcm_hit_where * 16/1000, lcm_wvfm[lcm_hit_where], 'x', color='blue', label='LCM hits')

    # true hits
    true_hits_tpc_evt = true_hits[(true_hits['event_id'] == i_evt) & (true_hits['tpc_num'] == tpc)]['start_time_idx'].values
    true_hits_idx = np.array(true_hits_tpc_evt)
    for hit_idx in true_hits_idx:
      label = 'True hits' if hit_idx == true_hits_idx[0] else None
      axs[tpc].axvline(hit_idx * 16/1000, color='green', linestyle=':', linewidth=2, label=label)

    axs[tpc].set_xlabel('Time (us)')
    axs[tpc].set_ylabel('SPEs')
    axs[tpc].set_title(f'TPC {tpc}')

    # legend
    axs[tpc].legend()

  plt.tight_layout()
  plt.show()


# plot summed traptype waveforms per tpc with interactions
i_evt_lrs = 2
plot_summed_trap_waveform_per_tpc_with_interactions(i_evt_lrs, time_bins, summed_wvfms, hits, true_hits)

# plot summed traptype waveforms per tpc with interactions
i_evt_lrs = 25
plot_summed_trap_waveform_per_tpc_with_interactions(i_evt_lrs, time_bins, summed_wvfms, hits, true_hits)

In [12]:
def plot_all_tpc_waveform(wvfms, time_bins, hit_where, i_evt_lrs, i_mask, true_hits=None):

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
  # set a title
  fig.suptitle(f'Event {i_evt_lrs}, TrapType {i_mask % 2}')

  # Linear plot
  for i,wvfm in enumerate(wvfms):
    ax1.plot(time_bins, wvfm, label=f'TPC {i}')
    label = 'Rec hits' if i == 0 else None
    ax1.plot(hit_where[i] * 16/1000, wvfm[hit_where[i]], 'x', color='k', label=label)
  for hit_idx in true_hits:
    label = 'True hits' if i == 0 else None
    ax1.axvline(hit_idx * 16/1000, color='purple', linestyle=':', alpha=0.5, label=label)
  ax1.axhline(0, color='k', linestyle=':', alpha=0.5)
  ax1.set_xlabel('Time (us)')
  ax1.set_ylabel('SPEs')
  ax1.legend()

  # Logarithmic plot
  for i,wvfm in enumerate(wvfms):
    ax2.plot(time_bins, wvfm + 1, label=f'TPC {i}')
    label = 'Rec hits' if i == 0 else None
    ax2.plot(hit_where[i] * 16/1000, wvfm[hit_where[i]] + 1, 'x', color='k', label=label)
  for hit_idx in true_hits:
    label = 'True hits' if i == 0 else None
    ax2.axvline(hit_idx * 16/1000, color='purple', linestyle=':', alpha=0.5, label=label)
  ax2.axhline(1, color='k', linestyle=':', alpha=0.5)
  ax2.set_yscale('log')
  ax2.set_xlabel('Time (us)')
  #ax2.set_ylim(0.1, wvfm.max()*1.1)

  plt.tight_layout()
  plt.show()